In [2]:
%pip install keyring

  Using cached keyring-25.7.0-py3-none-any.whl.metadata (21 kB)
  Using cached jaraco.classes-3.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached jaraco_functools-4.4.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached more_itertools-10.8.0-py3-none-any.whl.metadata (39 kB)
  Using cached backports.tarfile-1.2.0-py3-none-any.whl.metadata (2.0 kB)
Using cached keyring-25.7.0-py3-none-any.whl (39 kB)
Using cached jaraco.classes-3.4.0-py3-none-any.whl (6.8 kB)
Using cached backports.tarfile-1.2.0-py3-none-any.whl (30 kB)
Using cached jaraco_functools-4.4.0-py3-none-any.whl (10 kB)
Using cached more_itertools-10.8.0-py3-none-any.whl (69 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [keyring]m5/6 [keyring]
Note: you may need to restart the kernel to use updated packages.


In [1]:
from treasury_ml_utils.snowflake_utils import SnowflakeConnector

snowflake_client = SnowflakeConnector("joshua.choy@transferwise.com")

 pip install snowflake-connector-python[secure-local-storage]


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://transferwise.okta-emea.com/app/snowflake/exk4istpb5gZUyV8u0i7/sso/saml?SAMLRequest=nZJbb%2BIwEIX%2FSuR9TpwEKNQCKgrLllUvCEKr7ZtJBrBw7KzHaYBfX4fLqvvQPvQtcs7Md2bOdG92ufTewKDQqkeiICQeqFRnQq17ZJGM%2FQ7x0HKVcakV9MgekNz0u8hzWbBBaTdqBn9LQOu5RgpZ%2FaNHSqOY5iiQKZ4DMpuy%2BeDhnsVByDgiGOtw5FySoXCsjbUFo7SqqqBqBNqsaRyGIQ2vqVPVkh%2FkA6L4mlEYbXWq5aVk52b6BBHRsFkjnMIRpufCW6FOK%2FiKsjyJkN0lydSfPs0T4g0u0w21wjIHMwfzJlJYzO5PBtA5mCVxuxl3Aij9FJQ1XPpRgEpXK8m3kOq8KK1rHLgvuoKMSr0Wbl2TUY8UW5EtN5l8HDd0IZdPw7vJMnpp%2FFz9fkhGcVkdDoPDr%2FVt%2B49s8V1UpcR7voQb1%2BFOEEuYqDpS657C%2BMoPIz9qJOEViyMWx0Grcf1KvJGLVChuj5UX386pwhWYSiAEemu5Dznwo01eFPTfBBR226ZAWyxb69fF%2FrlThqJNETWtsyOn82FHK6b%2FjaV06ccG52N8dPlMRlMtRbr3xtrk3H4eXxRExxeR%2BaujlEHOhRxkmQFEF6OUuhoa4NbdvDUlENo%2FUf%2

 pip install snowflake-connector-python[secure-local-storage]


9.40.7


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

In [6]:
t = "2025-11-01 00:00:00"
t1 = "2025-12-31 00:00:00"

In [7]:
# getting exposure data

event_df = snowflake_client.fetch(f"""
SELECT TO_TIMESTAMP(HOUR_TS) as HOUR_TS,
       CCY,
       ACCOUNT_TYPE,
       EVENT_TYPE,
       EXPOSURE_DIRECTION,
       SUM_EXPOSURE_AMOUNT
FROM ANALYTICS_DB.RPT_TREASURY.WEEKEND_FEATURES_EXPOSURE_EVENTS_HOURLY
WHERE HOUR_TS >= '{t}'
    AND HOUR_TS <= '{t1}'
    order by hour_ts
    limit 500;
""")

event_df.head()

,hour_ts,ccy,account_type,event_type,exposure_direction,sum_exposure_amount
0,2025-11-01,AED,CUSTOMER,PartnerExchangeRegisteredEvent,negative,-48780.50
1,2025-11-01,NPR,BORDERLESS,RollbackFixedRateGuaranteedEvent,positive,800426.00
2,2025-11-01,XOF,BORDERLESS,BalanceReserveConversionEvent,negative,-111000.00
3,2025-11-01,AUD,BORDERLESS,AcquiringPaymentPnlEvent,positive,13.68
4,2025-11-01,CZK,BORDERLESS,BalanceConversionPnlEvent,negative,-0.46


In [ ]:
# exogenous customer activity events 

exo_events = ['BalanceInstantConversionEvent', 'BalanceReserveConversionEvent', 'FixedTransferMarkReceivedEvent', 'FixedTransferRollbackReceivedEvent', 'FixedRatePromisedEvent', 
              'RollbackFixedRatePromisedEvent', 'PlasticTransactionEvent']


In [43]:
sorted(event_df.event_type.unique())

['AcquiringPaymentConfirmedEvent',
 'AcquiringPaymentPnlEvent',
 'BalanceConversionPnlEvent',
 'BalanceInstantConversionEvent',
 'BalanceReleaseReservedEvent',
 'BalanceReserveConversionEvent',
 'BalanceTransactionV2PnlEvent',
 'BankTransactionPnlEvent',
 'CustomerTransferPnlEvent',
 'DirectDebitBalanceTransactionLinkedV2Event',
 'ExtraMoneyPnlEvent',
 'FixedRateGuaranteedEvent',
 'FixedRatePromisedEvent',
 'FixedTransferMarkReceivedEvent',
 'FixedTransferPayoutAllocatedEvent',
 'FixedTransferPayoutRollbackAllocatedEvent',
 'FixedTransferRollbackReceivedEvent',
 'JournalCustomerTransferAsLossEvent',
 'PartnerExchangeRegisteredEvent',
 'PayoutRestartedEvent',
 'PayoutStoppedEvent',
 'PlasticPayoutsV2PnlEvent',
 'PlasticTransactionCreatedEvent',
 'ResetCustomerTransferBalanceEvent',
 'RollbackFixedRateGuaranteedEvent',
 'RollbackFixedRatePromisedEvent',
 'RollbackPlasticTransactionCreatedEvent',
 'SchemeSettlementPnlEvent',
 'TransferCancelledEvent']

In [10]:
# exposure delta

delta = snowflake_client.fetch(f"""

        SELECT
            account.CCY AS ccy,
            TIME_SLICE(ledger.TIMESTAMP, 1, 'day', 'start') AS timestamp,
            SUM(ledger.AMOUNT) AS exposure_delta
        FROM RPT_TREASURY.LIQUIDITY_TRANSACTION_COMBINED AS ledger
        JOIN LIQUIDITY.ACCOUNT AS account
        ON ledger.ACCOUNTID = account.ID
        LEFT JOIN RPT_TREASURY.RAW_LIQUIDITY__SUBACCOUNT_MAPPING AS account_mapping
        ON ledger.SUBACCOUNTTYPE = account_mapping.SUBACCOUNT__TYPE
        WHERE TIMESTAMP > '{t}'
        AND TIMESTAMP <= '{t1}'
        AND CAUSINGEVENTTYPE NOT ILIKE '%MarketOrder%'
        AND account_mapping.SUBACCOUNT__GROUP != 'EQUITY'
        GROUP BY 1, 2
    """)

delta.head()

,ccy,timestamp,exposure_delta
0,SZL,2025-11-30,-1845.00
1,USD,2025-11-02,62428858.44
2,ALL,2025-11-02,-455533.93
3,NOK,2025-11-02,1463536.40
4,RSD,2025-11-02,-2022890.06


In [32]:
delta["timestamp"] = pd.to_datetime(delta["timestamp"])
delta_df = delta.pivot(index="timestamp", columns="ccy", values="exposure_delta")
delta_df.sort_values("timestamp")

ccy,AED,AFN,ALL,AMD,AOA,ARS,AUD,AWG,AZN,BAM,...,VUV,WST,XAF,XCD,XCG,XOF,XPF,YER,ZAR,ZMW
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-11-01,-24647656.27,NaN,-236740.68,-7913644.31,396520.16,-1.476135e+08,44906421.77,40.00,-27717.73,-36468.54,...,-484598.0,-3835.56,-415203.0,-6235.05,314.09,-3040039.0,1001464.0,NaN,-3.153287e+07,-28949.24
2025-11-02,-29354554.43,NaN,-455533.93,-7098156.91,487011.47,1.198378e+06,36094557.14,0.00,-23727.23,16880.89,...,-696496.0,-3613.45,-578444.0,-6038.17,152.59,-1329235.0,-1145079.0,NaN,-2.482965e+07,100512.31
2025-11-03,-78855664.79,NaN,1463065.94,14844199.97,-1664711.03,-1.976358e+08,49054115.73,0.00,55185.18,27623.23,...,1105621.0,13332.96,-380485.0,-21612.25,-382.92,2988332.0,1053317.0,NaN,-1.118966e+08,-48778.74
2025-11-04,-76385752.25,NaN,-207477.63,1559902.10,1898338.56,9.380150e+07,2659018.38,0.00,6421.79,-5174.79,...,70383.0,2194.03,-1771544.0,34498.73,518.25,1251984.0,1160806.0,NaN,-4.349459e+07,34296.79
2025-11-05,-87805913.84,NaN,199810.37,107035.52,-688553.66,-1.863792e+06,17287768.84,-182.45,-903.81,6066.98,...,63119.0,-4546.94,421382.0,-12149.93,729.44,-1512711.0,-2289459.0,NaN,-7.792714e+07,-80104.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,-24303992.86,NaN,-249734.87,-6765965.75,-978120.89,-1.957676e+07,49571687.87,-559.25,-53122.56,-30569.21,...,-2024442.0,-3411.32,1942792.0,-6601.65,271.71,-960006.0,-3027752.0,-1188.32,-2.159400e+07,-148481.11
2025-12-28,-24211401.31,NaN,-425801.70,-7963240.07,2141635.30,-3.843587e+08,39882761.67,720.78,-13814.44,7632.09,...,-1505740.0,-20533.09,-65811.0,2489.78,698.97,-1865511.0,-2497458.0,1188.32,-2.545585e+07,124910.31
2025-12-29,-94151268.49,NaN,1122064.66,16122214.13,246313.03,-3.035784e+07,28767177.96,14.10,74514.19,12466.55,...,7475460.0,15002.38,2248865.0,-21343.65,-240.28,1218329.0,5830231.0,NaN,-8.241392e+07,-176100.01


In [23]:
delta.loc[delta['timestamp']=="2025-11-30"]

,ccy,timestamp,exposure_delta
0,SZL,2025-11-30,-1.845000e+03
52,KZT,2025-11-30,3.770309e+06
53,GNF,2025-11-30,-1.014000e+05
92,CHF,2025-11-30,6.137366e+06
93,HKD,2025-11-30,9.044863e+06
...,...,...,...
7939,NZD,2025-11-30,1.432926e+07
7940,VND,2025-11-30,-7.802826e+10
7941,NIO,2025-11-30,1.362131e+05
7942,OMR,2025-11-30,-1.459620e+03


In [ ]:
# getting ccy data

df = snowflake_client.fetch(f"""
    SELECT *, ((BID_OUTRIGHT+OFFER_OUTRIGHT)/2) as mid
    FROM RPT_TREASURY.INT_ASSET_PRICING__FX_QUOTE_SNAPSHOT_HOURLY
    WHERE CCY_PAIR IN ('USDPHP', 'EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD', 'NZDUSD','USDCAD', 'USDCNH', 'USDSGD')
    AND TENOR_CODE = 'SP'
    AND INSTRUMENT_TYPE = 'FXSpot'
    AND PROVIDER = 'BLOOMBERG'
    AND TIMESTAMP BETWEEN '{t}' AND '{t1}'
""")
ccy_df = df.pivot(index='timestamp', columns='ccy_pair', values='mid')
ccy_df.head()

ccy_pair,AUDUSD,EURUSD,GBPUSD,NZDUSD,USDCAD,USDCNH,USDJPY,USDPHP,USDSGD
timestamp,,,,,,,,,
2026-01-11 00:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
2026-01-11 01:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
2026-01-11 02:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
2026-01-11 03:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
2026-01-11 04:00:00,0.6687,1.1636,1.3404,0.57315,1.39135,6.97595,157.89,59.2565,1.28745
